In [21]:
%pip install nvidia-nvcomp-cu12

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import cupy as cp
from nvidia import nvcomp

def encode_file(input_file_path, output_file_path, algorithm_type):
  with open(input_file_path, "rb") as f:
    text = f.read()

  nvarr_txt_h = nvcomp.as_array(text)

  # Transfer the data to the GPU memory
  nvarr_txt_d = nvarr_txt_h.cuda()
  # Create the Lz4 codec object with checksum verification enabled
  lz4_codec = nvcomp.Codec(algorithm=algorithm_type, checksum_policy=nvcomp.ChecksumPolicy.COMPUTE_AND_VERIFY)
  # Perform the Lz4 compression on the GPU
  lz4_comp_arr = lz4_codec.encode(nvarr_txt_d)
  # Transfer the compressed data back to the host (CPU)
  compressed_data = lz4_comp_arr.cpu()
  # Write the compressed data to the output file in binary mode
  with open(output_file_path, "wb") as f:
    f.write(compressed_data)

  print(f"File '{input_file_path}' encoded successfully to '{output_file_path}'.")

# Example usage
input_file = "/home/arpan/codes/Encrypt5/Encrypt5_SecurityProtocols/GPU_compression/lage_file.txt"
output_file = "/home/arpan/codes/Encrypt5/Encrypt5_SecurityProtocols/GPU_compression/large_file.LZ4"
encode_file(input_file, output_file, "LZ4")

FileNotFoundError: [Errno 2] No such file or directory: '/home/arpan/codes/Encrypt5/Encrypt5_SecurityProtocols/GPU_compression/lage_file.txt'

In [1]:
import numpy as np
import cupy as cp
from nvidia import nvcomp

print("nvcomp version:", nvcomp.__version__)
print("nvcomp cuda version:", nvcomp.__cuda_version__)

def verify(original_file, encoded_file, algorith_type):
    with open(original_file, "rb") as f:
        original = f.read()
    with open(encoded_file, "rb") as f:
        text = f.read()

    nvarr_txt_h = nvcomp.as_array(text)
    nvarr_txt_d = nvarr_txt_h.cuda()
    
    lz4_codec = nvcomp.Codec(algorithm=algorith_type, checksum_policy=nvcomp.ChecksumPolicy.COMPUTE_AND_VERIFY)
    nv_dec_array = lz4_codec.decode(text)

    is_equal = bytes(nv_dec_array.cpu()) == bytes(original)
    print("Is decoded equal to original?", is_equal)
    return is_equal

def decode(encoded_file, algorithm_type):
    with open(encoded_file, "rb") as f:
        text = f.read()

    nvarr_txt_h = nvcomp.as_array(text)
    nvarr_txt_d = nvarr_txt_h.cuda()

    lz4_codec = nvcomp.Codec(algorithm=algorithm_type, checksum_policy=nvcomp.ChecksumPolicy.COMPUTE_AND_VERIFY)
    nv_dec_array = lz4_codec.decode(text)

    print(bytes(nv_dec_array.cpu())[:400].decode())

verify("mobydick.txt", "mobydick.LZ4", "LZ4")
decode("mobydick.LZ4", "LZ4")


nvcomp version: 4.0.1
nvcomp cuda version: 12050


FileNotFoundError: [Errno 2] No such file or directory: 'mobydick.LZ4'

In [ ]:
import GPUtil
import numpy as np
import cupy as cp
from nvidia import nvcomp

print("nvcomp version:", nvcomp.__version__)
print("nvcomp cuda version:", nvcomp.__cuda_version__)

def get_gpu_memory_info():
    GPUs = GPUtil.getGPUs()
    gpu = GPUs[0]  # Assuming you have one GPU
    return gpu.memoryTotal, gpu.memoryUsed, gpu.memoryFree

# Example usage:
total_memory = get_gpu_memory_info()[0]
print(f"Total GPU memory: {total_memory} MB")




Total GPU memory: 4096.0 MB


In [19]:
import os

def get_file_size_mb(file_path):

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    size_bytes = os.path.getsize(file_path)
    size_mb = size_bytes / (1024 * 1024)
    return size_mb

# Example usage:
file_path = "mobydick.txt"  # Replace with the actual file path
file_size_mb = get_file_size_mb(file_path)
print(f"File size: {file_size_mb:.2f} MB")

File size: 0.31 MB


In [4]:
import os

def generate_large_text_file(filename, size_in_mb):
    # Define how many bytes 1 MB has
    bytes_in_mb = 1024 * 1024
    
    # Calculate how many bytes the file should have
    target_size_in_bytes = size_in_mb * bytes_in_mb
    
    # Sample text to write (you can modify this as needed)
    sample_text = "This is a sample UTF-8 encoded text. " \
                  "You can add more variety to the text as needed. " \
                  "Here's some unicode: 😊, 🐍, 🚀\n"

    # UTF-8 encoding can vary in byte size per character, so we repeatedly append
    encoded_sample = sample_text.encode('utf-8')

    # Create the file and write until the size is reached
    with open(filename, 'wb') as file:
        current_size = 0
        
        while current_size < target_size_in_bytes:
            file.write(encoded_sample)
            current_size += len(encoded_sample)
    
    print(f"File '{filename}' of size {size_in_mb} MB generated successfully!")

# Example usage
filename = "/home/warmachine/codes/Encrypt5/Encrypt5_SecurityProtocols/GPU_compression/large_file.txt"
size_in_mb = 1000 # Set the desired file size in MB

generate_large_text_file(filename, size_in_mb)

File '/home/warmachine/codes/Encrypt5/Encrypt5_SecurityProtocols/GPU_compression/large_file.txt' of size 1000 MB generated successfully!


In [ ]:
def txt_to_bin(txt_file, bin_file):
    try:
        # Open the text file in read mode and binary file in write mode
        with open(txt_file, 'r') as txt, open(bin_file, 'wb') as bin:
            # Read the content of the text file
            content = txt.read()
            
            # Encode the content to binary (UTF-8 encoding)
            binary_data = content.encode('utf-8')
            
            # Write the binary data to the .bin file
            bin.write(binary_data)
        
        print(f"Successfully converted {txt_file} to {bin_file}")
    
    except Exception as e:
        print(f"Error: {e}")

# Example usage
txt_file = 'large_file.txt'  # Path to your .txt file
bin_file = 'output.bin'   # Desired output .bin file

txt_to_bin(txt_file, bin_file)


In [ ]:
import os

def split_file(file_path, num_parts):
    # Get the file size
    file_size = os.path.getsize(file_path)
    
    # Calculate the size of each part
    part_size = file_size // num_parts
    leftover = file_size % num_parts

    # Open the file in binary read mode
    with open(file_path, 'rb') as f:
        for i in range(num_parts):
            # Determine the part file name
            part_file_name = f'{file_path}.part{i+1}'
            
            # Determine the size for this part (add leftover to the last part)
            current_part_size = part_size + (leftover if i == num_parts - 1 else 0)
            
            # Read part of the file
            part_data = f.read(current_part_size)
            
            # Write part to a new file
            with open(part_file_name, 'wb') as part_file:
                part_file.write(part_data)
            
            print(f"Part {i+1} created: {part_file_name}, Size: {current_part_size} bytes")

def merge_files(part_files, output_file_path):
    # Open the output file in binary write mode
    with open(output_file_path, 'wb') as output_file:
        for part_file in part_files:
            # Read each part and append it to the output file
            with open(part_file, 'rb') as pf:
                output_file.write(pf.read())
            
            print(f"Merged: {part_file}")
    
    print(f"All parts merged into {output_file_path}")


# Example Usage
# To split a 4GB file into 4 parts:
#split_file('large_file.txt', 2)

# To merge the files back:
merge_files(['large_file.txt.part1', 'large_file.txt.part2'], 'mergedfile.txt')
